In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


In [49]:
dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv')
dataset1 = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-cleaning/final_dataset_first_cleaning.csv')
dataset2= pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-cleaning/final_dataset_cleaned_kbest.csv')
dataset3 = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-cleaning/final_dataset_cleaned_random_forest_feature_selector.csv')

dataset.drop(['Open_time'], axis=1, inplace=True)
dataset.drop(['Sentimiento'], axis=1, inplace=True)
dataset.drop(['Sentimiento_coin'], axis=1, inplace=True)
dataset.drop(['Sentimiento_referentes'], axis=1, inplace=True)

dataset1.drop(['Open_time'], axis=1, inplace=True)
dataset2.drop(['Open_time'], axis=1, inplace=True)
dataset3.drop(['Open_time'], axis=1, inplace=True)


In [50]:
def basic_logistic_regression(dataset):
    # Dividir los datos en conjunto de entrenamiento y conjunto de prueba
    X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns=["Tendencia"]), 
                                                        dataset["Tendencia"], 
                                                        test_size=0.2, 
                                                        random_state=42)

    # Entrenar un modelo final de Regresión Logística utilizando las características seleccionadas
    final_model = LogisticRegression()
    final_model.fit(X_train, y_train)

    # Evaluar el modelo final
    accuracy = final_model.score(X_test, y_test)
    return accuracy


In [51]:
def basic_gradient_boosting(dataset):
    X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns=["Tendencia"]), 
                                                        dataset["Tendencia"], 
                                                        test_size=0.2, 
                                                        random_state=42)

    # Crear el clasificador GradientBoostingClassifier
    gb_model = GradientBoostingClassifier(
        n_estimators=1000,  # Usar 1000 estimadores
        learning_rate=0.1,  # Tasa de aprendizaje
        max_depth=5,  # Profundidad máxima de cada árbol
        min_samples_split=2,  # Número mínimo de muestras requeridas para dividir un nodo interno
        min_samples_leaf=1,  # Número mínimo de muestras requeridas para estar en una hoja
        subsample=0.8,  # Fracción de muestras a utilizar para ajustar los estimadores base
        max_features='sqrt',  # Número máximo de características a considerar al dividir nodos: raíz cuadrada del número de características
        random_state=42
    )

    # Entrenar el modelo
    gb_model.fit(X_train, y_train)

    # Evaluar el modelo
    accuracy = gb_model.score(X_test, y_test)
    return accuracy


In [52]:
def basic_svm(dataset):
    X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns=["Tendencia"]), 
                                                        dataset["Tendencia"], 
                                                        test_size=0.2, 
                                                        random_state=42)

    svc_model = SVC(
        kernel='rbf',  # Kernel radial
        C=10.0,  # Parámetro de regularización
        gamma='scale',  # Coeficiente de kernel para 'rbf'
        random_state=42
    )

    # Entrenar el modelo
    svc_model.fit(X_train, y_train)

    # Evaluar el modelo
    accuracy = svc_model.score(X_test, y_test)

    return accuracy

In [53]:
def basic_MLP(dataset):
    X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns=["Tendencia"]), 
                                                        dataset["Tendencia"], 
                                                        test_size=0.2, 
                                                        random_state=42)

    mlp_model = MLPClassifier(
        hidden_layer_sizes=(100, 50),  # Dos capas ocultas con 100 y 50 neuronas respectivamente
        activation='relu',  # Función de activación ReLU
        solver='adam',  # Optimizador Adam
        alpha=0.0001,  # Tasa de regularización L2
        learning_rate='adaptive',  # Tasa de aprendizaje adaptativa
        max_iter=1000,  # Número máximo de iteraciones
        random_state=42
    )

    # Entrenar el modelo
    mlp_model.fit(X_train, y_train)

    # Evaluar el modelo
    accuracy = mlp_model.score(X_test, y_test)
    return accuracy

In [54]:
def predict_constant_class(y_true, constant_class):
    y_pred_constant = [constant_class] * len(y_true)
    
    # Calcular el accuracy entre las etiquetas verdaderas y las predicciones constantes
    accuracy = accuracy_score(y_true, y_pred_constant)
    
    return accuracy

In [55]:
print("Exactitud del modelo final regresion logistica 0:", basic_logistic_regression(dataset))
print("Exactitud del modelo final regresion logistica 1:", basic_logistic_regression(dataset1))
print("Exactitud del modelo final regresion logistica 2:", basic_logistic_regression(dataset2))
print("Exactitud del modelo final regresion logistica 3:", basic_logistic_regression(dataset3))

print("Exactitud del modelo final gradient boosting 0:", basic_gradient_boosting(dataset))
print("Exactitud del modelo final gradient boosting 1:", basic_gradient_boosting(dataset1))
print("Exactitud del modelo final gradient boosting 2:", basic_gradient_boosting(dataset2))
print("Exactitud del modelo final gradient boosting 3:", basic_gradient_boosting(dataset2))

print("Exactitud del modelo final SVM 0:", basic_svm(dataset))
print("Exactitud del modelo final SVM 1:", basic_svm(dataset1))
print("Exactitud del modelo final SVM 2:", basic_svm(dataset2))
print("Exactitud del modelo final SVM 3:", basic_svm(dataset3))

print("Exactitud del modelo final MLP 0:", basic_MLP(dataset))
print("Exactitud del modelo final MLP 1:", basic_MLP(dataset1))
print("Exactitud del modelo final MLP 2:", basic_MLP(dataset2))
print("Exactitud del modelo final MLP 3:", basic_MLP(dataset3))

print("Exactitud clase constante - Alcista fuerte:", predict_constant_class(dataset['Tendencia'], "Alcista fuerte"))
print("Exactitud clase constante - Alcista fuerte:", predict_constant_class(dataset['Tendencia'], "Alcista fuerte"))
print("Exactitud clase constante - Alcista leve:", predict_constant_class(dataset['Tendencia'], "Alcista leve"))
print("Exactitud clase constante - Lateral:", predict_constant_class(dataset['Tendencia'], "Lateral"))
print("Exactitud clase constante - Bajista leve:", predict_constant_class(dataset['Tendencia'], "Bajista leve"))
print("Exactitud clase constante - Bajista fuerte:", predict_constant_class(dataset['Tendencia'], "Bajista fuerte"))

/Users/mmarchetta/Desktop/Tesis-2024/myenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mmarchetta/Desktop/Tesis-2024/myenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable

Exactitud del modelo final regresion logistica 0: 0.24571428571428572
Exactitud del modelo final regresion logistica 1: 0.3657142857142857
Exactitud del modelo final regresion logistica 2: 0.37714285714285717


/Users/mmarchetta/Desktop/Tesis-2024/myenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Exactitud del modelo final regresion logistica 3: 0.35428571428571426
Exactitud del modelo final gradient boosting 0: 0.5085714285714286
Exactitud del modelo final gradient boosting 1: 0.48
Exactitud del modelo final gradient boosting 2: 0.4857142857142857
Exactitud del modelo final gradient boosting 3: 0.4857142857142857
Exactitud del modelo final SVM 0: 0.4
Exactitud del modelo final SVM 1: 0.46285714285714286
Exactitud del modelo final SVM 2: 0.4057142857142857
Exactitud del modelo final SVM 3: 0.4057142857142857
Exactitud del modelo final MLP 0: 0.2342857142857143
Exactitud del modelo final MLP 1: 0.24571428571428572
Exactitud del modelo final MLP 2: 0.2742857142857143
Exactitud del modelo final MLP 3: 0.25142857142857145
Exactitud clase constante - Alcista fuerte: 0.2227324913892078
Exactitud clase constante - Alcista fuerte: 0.2227324913892078
Exactitud clase constante - Alcista leve: 0.18714121699196326
Exactitud clase constante - Lateral: 0.1894374282433984
Exactitud clase cons